## 1. Build a ChatBot pipeline Using Hugging Face Transformers Library

Install the transformers library and import the pipeline method to use the open source models also add an additional library to avoid warnings message.

In [27]:
!pip intall transformers

ERROR: unknown command "intall" - maybe you meant "install"


In [28]:
from transformers import pipeline
from transformers.utils import logging
logging.set_verbosity_error()

Define the conversation pipeline. The **pipeline()** method in the Transformers library is used to create a simple pipeline for executing a sequence of tasks. First you need to specify the task and then the model you want to use. When you call pipeline() with these parameters, it creates a pipeline object that you can use to analyze text and perform sentiment analysis using the specified model.

In [29]:
chatbot = pipeline(task="conversational",
                   model="facebook/blenderbot-400M-distill")

Info about ['blenderbot-400M-distill'](https://huggingface.co/facebook/blenderbot-400M-distill)

In [30]:
user_message = """
Suggest me some good project ideas on machine learning?
"""

In [31]:
from transformers import Conversation

In [32]:
conversation = Conversation(user_message)

In [33]:
print(conversation)

Conversation id: eb7c0de9-f29a-412f-b8b2-af3eb8055eb6
user: 
Suggest me some good project ideas on machine learning?




In [34]:
conversation = chatbot(conversation)

In [35]:
print(conversation)

Conversation id: eb7c0de9-f29a-412f-b8b2-af3eb8055eb6
user: 
Suggest me some good project ideas on machine learning?

assistant:  I'm not sure, but I do know that machines are used in many fields of science and technology.



- You can continue the conversation with the chatbot with:
```
print(chatbot(Conversation("What else do you recommend?")))
```
- However, the chatbot may provide an unrelated response because it does not have memory of any prior conversations.

- To include prior conversations in the LLM's context, you can add a 'message' to include the previous chat history.

In [36]:
conversation.add_message(
    {"role": "user",
     "content": """
What else do you recommend?
"""
    })

In [37]:
print(conversation)

Conversation id: eb7c0de9-f29a-412f-b8b2-af3eb8055eb6
user: 
Suggest me some good project ideas on machine learning?

assistant:  I'm not sure, but I do know that machines are used in many fields of science and technology.
user: 
What else do you recommend?




In [38]:
conversation = chatbot(conversation)
print(conversation)

Conversation id: eb7c0de9-f29a-412f-b8b2-af3eb8055eb6
user: 
Suggest me some good project ideas on machine learning?

assistant:  I'm not sure, but I do know that machines are used in many fields of science and technology.
user: 
What else do you recommend?

assistant:  Well, I know that there are many different types of machines that can be used. 



- [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- [LMSYS Chatbot Arena Leaderboard](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard)

## 2. Translation and Summarization

In [39]:
import torch

In [40]:
translator = pipeline(task="translation",
                      model="facebook/nllb-200-distilled-600M",
                      torch_dtype=torch.bfloat16)

NLLB: No Language Left Behind: ['nllb-200-distilled-600M'](https://huggingface.co/facebook/nllb-200-distilled-600M). <br>
This model is capable of translating 200 languages. By setting the dtype to BFloat16 we are able to compress the model without any performance degradation.

In [41]:
text = """\
My pet is adorable, \
Your pet is cute.
Her pet is friendly.
His pet is thoughtful. \
We all have nice pets!"""

After creating the pipeline object we can pass the text that we want to translate and we can set the source language (here english) and the language in which we want the translation (here french).

In [42]:
text_translated = translator(text,
                             src_lang="eng_Latn",
                             tgt_lang="fra_Latn")

To choose other languages, you can find the other language codes on the page: [Languages in FLORES-200](https://github.com/facebookresearch/flores/blob/main/flores200/README.md#languages-in-flores-200)

For example:
- Afrikaans: afr_Latn
- Chinese: zho_Hans
- Egyptian Arabic: arz_Arab
- French: fra_Latn
- German: deu_Latn
- Greek: ell_Grek
- Hindi: hin_Deva
- Indonesian: ind_Latn
- Italian: ita_Latn
- Japanese: jpn_Jpan
- Korean: kor_Hang
- Persian: pes_Arab
- Portuguese: por_Latn
- Russian: rus_Cyrl
- Spanish: spa_Latn
- Swahili: swh_Latn
- Thai: tha_Thai
- Turkish: tur_Latn
- Vietnamese: vie_Latn
- Zulu: zul_Latn

In [43]:
text_translated

[{'translation_text': 'Mon animal est adorable, ton animal est mignon, son animal est ami, son animal est attentionné, nous avons tous de beaux animaux.'}]

### Free up some space
In order to have enough free memory to run the rest of the code, please run the following to free up memory on the machine.

In [44]:
import gc
del translator
gc.collect()

10217

In [45]:
summarizer = pipeline(task="summarization",
                      model="facebook/bart-large-cnn",
                      torch_dtype=torch.bfloat16)

In [46]:
text = """Indian history is a tapestry of diverse cultures, religions, and
civilizations that have thrived on the Indian subcontinent for thousands of years.
From the ancient civilizations of the Indus Valley and Vedic period to the Maurya and
Gupta empires, India has been home to numerous great empires, dynasties, and cultures
that have left a profound impact on its history. The country has been a cradle of major
religions such as Hinduism, Buddhism, Jainism, and Sikhism, each contributing to its rich
cultural heritage. India has also been a melting pot of cultures, with influences from
Central Asia, Persia, and Europe shaping its history. The Mughal Empire, known for its
architectural marvels and cultural synthesis, and the British colonial rule, which
significantly shaped modern India, are key chapters in its history. Today, India stands
as a vibrant democracy and a testament to its rich history and cultural diversity."""

To summarize the above paragraph just pass the text into the summarizer pipeline and set the minimum and maximum length of the summary.

In [47]:
summary = summarizer(text,
                     min_length=10,
                     max_length=100)

In [48]:
summary

[{'summary_text': 'Indian history is a tapestry of diverse cultures, religions, andcivilizations that have thrived on the Indian subcontinent for thousands of years. The country has been a cradle of major majorreligions such as Hinduism, Buddhism, Jainism, and Sikhism. India has also been a melting pot of cultures, with influences from Central Asia, Persia, and Europe shaping its history.'}]

In [49]:
!pip install gradio
import gradio as gr

In [50]:
message_list = []
response_list = []

def chatbot(message, history):
    conversation = chatbot(message)

    return conversation[0]['generated_text']

demo_chatbot = gr.ChatInterface(chatbot, title="Chatbot", description="Enter text to start chatting.")

demo_chatbot.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ca1dd447b276db3241.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
